In [88]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [89]:
df = pd.read_csv('dataset/processed_english_twitter_data.csv', skipinitialspace=True, engine="python")
df.head()

,Lang,Text,Author,Service
0,en,card support good yet sebastian release neckla...,xYouCanCallMeAl,card
1,en,rt terraloire big bank multiple leader sit hos...,MarttinenLee,investment
2,en,rt wisdomise wisdomise main goal make financia...,Cryptullo,investment
3,en,hightoweradvsor wealth management firm based c...,AccountingToday,investment
4,en,rt warpsiwah cast emi nishino full movie http ...,Kakeksugion01,emi


In [91]:
df.groupby(by='Service').count()

,Lang,Text,Author
Service,,,
card,483,483,483
emi,92,92,92
insurance,485,485,485
investment,149,149,149
loan,421,421,421
savings,106,106,106


Array - [complaint, enquiry, general talk]

In [95]:
import pickle as pkl

In [96]:
intent=pkl.load(open('model/intent_classification.pkl','rb'))
intent_tfidf=pkl.load(open('model/intent_classification_tfidf.pkl','rb'))

s=['I want to buy a good insurance! Suggest me a good company!']
intent.predict(intent_tfidf.transform(s))

C:\Users\ayush\AppData\Local\Temp\ipykernel_7800\785994648.py:2: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  intent_tfidf=pkl.load(open('model/intent_classification_tfidf.pkl','rb'))


array([[1, 0, 0]])

* All zeroes - Anger
* 1 0 0 0 0 - Fear
* 0 1 0 0 0 - Joy
* 0 0 1 0 0 - Neutral
* 0 0 0 1 0 - Sad
* 0 0 0 0 1 - Happy

In [97]:
emotion=pkl.load(open('model/emotion_multiclass_model.pkl','rb'))
emotion_tfidf=pkl.load(open('model/emotion_multiclass_model_tfidf.pkl','rb'))

s=['All the insurance companies are fraudulent! I am fed up with them.']
emotion.predict(emotion_tfidf.transform(s))

d:\Projects\Web scraping and Automation\env\lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.0.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
d:\Projects\Web scraping and Automation\env\lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator LabelBinarizer from version 1.0.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
d:\Projects\Web scraping and Automation\env\lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator OneVsRestClassifier from v

array([[0, 0, 0, 0, 0]])

0 - Negative sentiment <br>
1 - Positive sentiment

In [100]:
sentiment = pkl.load(open('model/tweet_clf.pkl','rb'))
sentiment_tfidf = pkl.load(open('model/tweet_tfidf.pkl','rb'))

s=['These insurance companies are a huge headache. I cannot deal with them anymore!']
sentiment.predict(sentiment_tfidf.transform(s))

d:\Projects\Web scraping and Automation\env\lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator LinearSVC from version 1.0.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\ayush\AppData\Local\Temp\ipykernel_7800\1596978838.py:2: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  sentiment_tfidf = pkl.load(open('model/tweet_tfidf.pkl','rb'))
d:\Projects\Web scraping and Automation\env\lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 1.0.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org

array([0], dtype=int64)